In [1]:
# import libraries
import uproot3 as uproot
import uproot as uproot4
import numpy as np
import numba
import awkward as ak
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import                                                                                                                  
import mplhep as hep
plt.style.use(hep.style.ROOT)
import os,sys
import pandas as pd
from scipy.stats import halfnorm
from sklearn.cluster import KMeans
import random
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from scipy.optimize import curve_fit
from scipy.stats import crystalball
from lmfit import Model
import copy
import math
from tqdm import tqdm

#import ROOT
#from coffea import hist

In [2]:
# Process Data
def getData(fname="", procName="Events"):
    kn_dict = uproot.open(fname)["Events"].arrays()
    kn_dict_ak1 = {name.decode(): ak.from_awkward0(array) for name, array in kn_dict.items()}
    kn_events = ak.zip({"Electrons":ak.zip({
                                            "ge":      kn_dict_ak1["ge"],
                                            "gvx":      kn_dict_ak1["gvx"],
                                            "gvy":      kn_dict_ak1["gvy"],
                                            "gvz":      kn_dict_ak1["gvz"],
                                            "gpx":      kn_dict_ak1["gpx"],
                                            "gpy":      kn_dict_ak1["gpy"],
                                            "gpz":      kn_dict_ak1["gpz"],
                                        }),
                        "Hits":ak.zip({
                                      "detID":   kn_dict_ak1["hit_detID"],
                                      "edep":    kn_dict_ak1["hit_edep"],
                                      "elmID":   kn_dict_ak1["hit_elmID"],
                                      "truthx":  kn_dict_ak1["hit_truthx"],
                                      "truthy":  kn_dict_ak1["hit_truthy"],
                                      "truthz":  kn_dict_ak1["hit_truthz"],
                                      "hit_pos":  kn_dict_ak1["hit_pos"],
                                      }),
                        "Cal_Position":ak.zip({
                                            "track_x_CAL":      kn_dict_ak1["track_x_CAL"],
                                            "track_y_CAL":     kn_dict_ak1["track_y_CAL"],
                                        }),
                        "coordinates":ak.zip({
                                        "sx_ecal":   kn_dict_ak1["sx_ecal"],
                                        "sy_ecal":   kn_dict_ak1["sy_ecal"],
                                        "sz_ecal":   kn_dict_ak1["sz_ecal"],
                            
                                        }),
                            
                    
                       }, depth_limit=1)
    return kn_events

In [3]:
def emcal_simhit_selection(arr):
    mask = (arr.detID == 100)
    return mask

def emcal_simhit_selection_energy(arr, e):
    mask = (arr.edep >= e)
    return mask

def h2_selection(arr):
    mask = (arr.detID >= 35) & (arr.detID <= 38)
    return mask

def st2_selection(arr):
    mask = (arr.detID >= 13) & (arr.detID <= 18)
    return mask

def st3_selection(arr):
    mask = (arr.detID >= 19) & (arr.detID <= 30)
    return mask

def h4_selection(arr):
    mask = (arr.detID >= 41) & (arr.detID <= 46)
    return mask

In [4]:
ntowersx=72
ntowersy=36
sizex=5.53 # in cm
sizey=5.53 # in cm
ecalx=[-200,200] #size in cm
ecaly=[-100,100]
binsx=ecalx[1]- ecalx[0]
binsy=ecaly[1]- ecaly[0]
sfc = 0.1146337964120158
emin=0.0005

In [5]:
def emcal_byevent(dq_events2,evtNum):
    dq_hits = dq_events2[evtNum]["Hits"]
    # select emcal hits
    emcal_mask = emcal_simhit_selection(dq_hits)
    emcal_hits = dq_hits[emcal_mask]
    emcal_energy_mask = emcal_simhit_selection_energy(emcal_hits, emin)
    emcal_hits = emcal_hits[emcal_energy_mask]
    #convert into coordinates and energy_dp
    emcal_towerx = emcal_hits.elmID//ntowersy
    emcal_towery = emcal_hits.elmID%ntowersy
    emcal_x = ecalx[0]+emcal_towerx*sizex
    emcal_y = ecaly[0]+emcal_towery*sizey
    emcal_edep = emcal_hits.edep
    return emcal_x, emcal_y, emcal_edep

In [6]:
def emcal_bytuple(file):
    dq_events1 = getData(file,"Events")
    x_pos = []#designed to be 2D
    y_pos = []#designed to be 2D
    eve_energy = []#designed to be 2D
    for i in range(len(dq_events1[:]["Hits"].edep)):
        output1=emcal_byevent(dq_events1, i)
        x_pos.append(output1[0])
        y_pos.append(output1[1])
        eve_energy.append(output1[2])
    return x_pos, y_pos, eve_energy
        

In [7]:
def gen_data(filename):
    output2=emcal_bytuple(filename)
    x_eve=output2[0]
    y_eve=output2[1]
    eng_eve=output2[2]
    wew_x = []
    wew_y = []
    for i in range(len(eng_eve)):
        eng_tot=sum(eng_eve[i])
        #x_bar=(np.einsum('i,i->i', x_eve[i], eng_eve[i]))/eng_tot
        #y_bar=(np.einsum('i,i->i', y_eve[i], eng_eve[i]))/eng_tot
        x_bar=(np.dot(x_eve[i], eng_eve[i]))/eng_tot
        y_bar=(np.dot(y_eve[i], eng_eve[i]))/eng_tot
        x_sq_eve = []
        y_sq_eve = []
        for j in range(len(eng_eve[i])):
            x_sq_eve.append(eng_eve[i][j]*(x_eve[i][j]-x_bar)**2)
            y_sq_eve.append(eng_eve[i][j]*(y_eve[i][j]-y_bar)**2)
        wew_x.append(math.sqrt(sum(x_sq_eve)/eng_tot))
        wew_y.append(math.sqrt(sum(y_sq_eve)/eng_tot))
    return wew_x, wew_y
            



In [8]:
def gen_plot(f_name):
    output3= gen_data(f_name)
    wew_x_out=output3[0]
    wew_y_out=output3[1]
    wew_x_stat=[0]*100
    wew_y_stat=[0]*100
    for i in range(len(wew_x_out)):
        wew_x_stat[round(wew_x_out[i])]+=1
        wew_y_stat[round(wew_y_out[i])]+=1
    print(len(wew_x_stat), wew_x_stat)
    print(len(wew_y_stat), wew_y_stat)

    fig = plt.figure(figsize=(10,8))
    plt.hist(wew_x_stat, bins=100, range=(0,100), histtype='step', density=False, color="blue")
    plt.show()
    fig = plt.figure(figsize=(10,8))
    plt.hist(wew_y_stat, bins=100, range=(0,100), histtype='step', density=False, color="blue")
    plt.show()

        
    

In [9]:
gen_plot("st23_electron_520_10000eve.root")

KeyboardInterrupt: 